In [2]:
# --- Setup: clone lab repo and enter it ---
!git clone https://github.com/dontolon/ml_beam_management_lab.git
%cd ml_beam_management_lab


Cloning into 'ml_beam_management_lab'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 24 (delta 4), reused 22 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (24/24), 1.38 MiB | 3.79 MiB/s, done.
Resolving deltas: 100% (4/4), done.
/home/dante/experiments/lab/notebooks/ml_beam_management_lab


/home/dante/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]
